In [15]:
import numpy as np

G = 6.674e-11
manually_set = 0     #set this to 1 if you want to give the values from terminal, otherwise read from input_raw.txt
starting_time = 0       #starting time added to the input_converted file
center_of_mass = 0      #set this to True if you want the coordinates to be in the center of mass

#mass is always set (we can change it in the future but I think it's the simplest way)
#now we are left with radius and velocity
set_radius = True       #if this is True the radius will be set to 1 ua (which can be changed in the code). Then the velocity will be calculated accordingly (to get G=1)
#if this is False the velocity will be set to 1000m/s and the radius will be calculated accordingly

mass = []
x = []
y = []
z = []
vx =[]
vy = []
vz = []
r = []

if manually_set == True:
    number_of_bodies = input("Enter the number of bodies:")
    for i in range(int(number_of_bodies)):
        mass.append(float(input("Enter mass:")))
        x.append(float(input("Enter x")))
        y.append(float(input("Enter y")))
        z.append(float(input("Enter z")))
        vx.append(float(input("Enter vx")))
        vy.append(float(input("Enter vy")))
        vz.append(float(input("Enter vz")))
    
        r.append(np.sqrt(x[i]**2 + y[i]**2 + z[i]**2))      #non so se serve
else:
    mass, x, y, z, vx, vy, vz = np.loadtxt("input_raw.txt", delimiter=" ", comments="#", dtype="float").T
    number_of_bodies = len(mass)

mass_x = [a * b for a, b in zip(mass, x)]      #mass times coordinate x
mass_y = [a * b for a, b in zip(mass, y)]      #mass times coordinate y
mass_z = [a * b for a, b in zip(mass, z)]      #mass times coordinate z
cm_x = np.sum(mass_x)/sum(mass)
cm_y = np.sum(mass_y)/sum(mass)
cm_z = np.sum(mass_z)/sum(mass)

mass_internal = mass[0]     #use the first mass as unit, you can choose another one

if set_radius == True:
    r_internal = 149597870700       #m, 1 ua, you can change this to be anything (but 0)
    v_internal = np.sqrt(G*mass_internal/r_internal)

else:
    v_internal = 1000       #m/s
    r_internal = G*mass_internal/v_internal**2




#print(v_internal)

new_mass = mass/mass_internal

if center_of_mass == True:
    new_x = (x-cm_x)/r_internal
    new_y = (y-cm_y)/r_internal
    new_z = (z-cm_z)/r_internal
else:
    new_x = x/r_internal
    new_y = y/r_internal
    new_z = z/r_internal

new_vx = vx/v_internal
new_vy = vy/v_internal
new_vz = vz/v_internal

with open("input_converted.txt", "w") as f:     #file ready to be given to the C code
    print(number_of_bodies, starting_time, sep="\n", file=f)
    np.savetxt(f, np.column_stack([new_mass, new_x, new_y, new_z, new_vx, new_vy, new_vz]), delimiter=" ")
    
with open("conversion_parameters.txt", "w") as f:
    print(mass_internal, r_internal, v_internal, sep="\n", file=f)
